In [1]:
# AGORA SOMENTE IREMOS REPLICAR TODOS OS PARÂMTROS DE VALIDAÇÃO PARA TESTE
# Primeiro passo è sempre importar ou instalar as bibliotecas necessárias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# Utilizaremos a biblioteca do 'pandas' para abrir os arquivos a serem analisados
# Dessa forma teremos uma primeira impressão do tipo de tratamento que deveremos fazer nas planilhas
# Cumprimento de linhas e colunas, variáveis dependentes e independentes, outliers do modelo, informações NOSQL, etc

test_data = pd.read_csv('test.csv')
test_data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,23,Private,33021,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K
2,21,Private,139722,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Puerto-Rico,<=50K
3,45,Local-gov,272182,Some-college,10,Married-civ-spouse,Tech-support,Husband,Black,Male,5013,0,40,United-States,<=50K
4,59,Private,158776,9th,5,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K


In [3]:
test_data.shape

(7327, 15)

In [4]:
# Primeiramente iremos separar as variáveis independentes da variável dependente
# X representa toda a tabela exeto pela última coluna ('income'), enquanto y representa a mesma

X = test_data.iloc[:,:-1]
y = test_data.iloc[:,-1]

In [ ]:
# Iremos verificar se dentro da tabela há alguma célula não preenchida com a seguinte fórmula

X.isnull().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
dtype: int64

In [6]:
# O mesmo se aplica para a variável y

y.isnull().sum()

np.int64(0)

In [ ]:
# Com isso verificamos que a tabela está sequencial, ou seja, não possui atributos nulos
# Mas, ainda assim, pode ser que haja atributos 'sem valor' (?) nos hiperparametros de 'workclass', 'occupation' e 'native-country'

# Substituiremos '?', além de 'Never-worked' e 'Without-pay' devido a baixa espressividade, por 'Outros'

X.loc[X['workclass'] == '?', 'workclass'] = 'Outros'
X.loc[X['workclass'] == 'Never-worked', 'workclass'] = 'Outros'
X.loc[X['workclass'] == 'Without-pay', 'workclass'] = 'Outros'
agrupado_workclass = X.groupby(['workclass']).size()
agrupado_workclass

workclass
Federal-gov          212
Local-gov            473
Outros               429
Private             5103
Self-emp-inc         248
Self-emp-not-inc     584
State-gov            278
dtype: int64

In [8]:
# Substituiremos '?' por 'Outros'

X.loc[X['occupation'] == '?', 'occupation'] = 'Outros'
agrupado_occupation = X.groupby(['occupation']).size()
agrupado_occupation

occupation
Adm-clerical         813
Armed-Forces           2
Craft-repair         929
Exec-managerial      910
Farming-fishing      234
Handlers-cleaners    313
Machine-op-inspct    413
Other-service        755
Outros               427
Priv-house-serv       32
Prof-specialty       913
Protective-serv      121
Sales                853
Tech-support         240
Transport-moving     372
dtype: int64

In [9]:
# Substituiremos '?' por 'Outros'

X.loc[X['native-country'] == '?', 'native-country'] = 'Outros'

#for i in X['native-country']:
#  if X.groupby(['native-country']).size()[i] < 500:
#    X.loc[X['native-country'] == i, 'native-country'] = 'Outros'

agrupado_country = X.groupby(['native-country']).size()
agrupado_country

native-country
Cambodia                         1
Canada                          30
China                           13
Columbia                        19
Cuba                            22
Dominican-Republic              13
Ecuador                          4
El-Salvador                     18
England                         24
France                           1
Germany                         31
Greece                           4
Guatemala                       10
Haiti                           12
Honduras                         2
Hong                             3
Hungary                          2
India                           24
Iran                            11
Ireland                          5
Italy                           16
Jamaica                         19
Japan                           20
Laos                             1
Mexico                         142
Nicaragua                        6
Outlying-US(Guam-USVI-etc)       2
Outros                         134
Peru 

In [10]:
# Iremos também retirar do modelo a coluna 'educational-num', ja que representam o mesmo valor, em forma numérica, da coluna 'education'

X.drop('educational-num', axis=1, inplace=True)
X

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,23,Private,33021,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
2,21,Private,139722,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Puerto-Rico
3,45,Local-gov,272182,Some-college,Married-civ-spouse,Tech-support,Husband,Black,Male,5013,0,40,United-States
4,59,Private,158776,9th,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7322,42,Private,173704,Some-college,Married-civ-spouse,Sales,Husband,White,Male,0,1887,50,United-States
7323,27,Private,220748,HS-grad,Never-married,Handlers-cleaners,Other-relative,Black,Male,0,0,48,United-States
7324,18,Private,186909,Some-college,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States
7325,44,Self-emp-not-inc,127482,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,7298,0,50,England


In [11]:
# Agora iremos transformar todos os atributos categóricos em atributos numéricos utilizando o Label Encoder

labelencoder = LabelEncoder()

for column in X.select_dtypes(include=['object']).columns:
  X[column] = labelencoder.fit_transform(X[column])

X

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,3,226802,1,4,6,3,2,1,0,0,40,38
1,23,3,33021,11,2,6,0,4,1,0,0,40,38
2,21,3,139722,11,4,2,1,4,1,0,0,40,32
3,45,1,272182,15,2,13,0,2,1,5013,0,40,38
4,59,3,158776,6,2,6,0,4,1,0,0,40,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7322,42,3,173704,15,2,12,0,4,1,0,1887,50,38
7323,27,3,220748,11,4,5,2,2,1,0,0,48,38
7324,18,3,186909,15,4,7,3,4,0,0,0,16,38
7325,44,5,127482,11,2,2,0,4,1,7298,0,50,8


In [12]:
# Iremos 'normalizar' os dados da coluna "fnlwgt", pois os mesmo, visualmente, destoam dos demais dados do DataFrame, e assim utilizaremos o StandScaler

sc = StandardScaler()
sc_tr = sc.fit_transform(X.iloc[:,2:3])
sc_tr

array([[ 0.3553278 ],
       [-1.49934796],
       [-0.4781139 ],
       ...,
       [-0.02648765],
       [-0.5952628 ],
       [ 0.85139215]])

In [13]:
# Substituiremos o "fnlwgt" pelos nossos dados ajustados "fnlwgt_ajustado" e excluiremos primeiro do DataFrame

X_ajustado = pd.concat([X, pd.DataFrame(sc_tr, columns=['fnlwgt_ajustado'])], axis=1)
X_ajustado.drop(columns=["fnlwgt"])
X_ajustado.head()

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,fnlwgt_ajustado
0,25,3,226802,1,4,6,3,2,1,0,0,40,38,0.355328
1,23,3,33021,11,2,6,0,4,1,0,0,40,38,-1.499348
2,21,3,139722,11,4,2,1,4,1,0,0,40,32,-0.478114
3,45,1,272182,15,2,13,0,2,1,5013,0,40,38,0.789659
4,59,3,158776,6,2,6,0,4,1,0,0,40,38,-0.295748


In [14]:
# Tambem iremos transformar a nossa variavel y com Label Encoder

y_encoded = labelencoder.fit_transform(y)
y_encoded

array([0, 0, 0, ..., 0, 1, 0])

In [15]:
# Agora que ja temos todo nosso DataFrame 'estruturado' da melhor forma, iremos treinar e testar os parâmetros

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_ajustado,y_encoded,test_size=0.3, random_state=0)

In [ ]:
# Após otimizarmos na medida do possível o tratamento de nosso DataFrame, tambem verificamos, segundo o TPOTClassifier, e de acordo com os parametros estabelecidos por nós,
# verificamos que o possível melhor modelo a ser utilizado é o "DecisionTreeClassifier", o qual utilizaremos a seguir

%pip install graphviz

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

In [17]:
# Criaremos o modelo de Decision Tree seguindo os parâmetros estabelecidos pelo TPOT

modelo = DecisionTreeClassifier(max_depth=8, min_samples_leaf=7, min_samples_split=14)
modelo.fit(X_treinamento,y_treinamento)

DecisionTreeClassifier(max_depth=8, min_samples_leaf=7, min_samples_split=14)

In [18]:
# Podemos tambem "imprimir" uma imagem de como seria a nossa árvore de decisão segundo o modelo

dot_data = export_graphviz(modelo, out_file=None, filled=True, feature_names=test_data.columns[:-1], class_names=True, rounded=True)

graph = graphviz.Source(dot_data)
graph.render("decision_tree", format="png")

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [19]:
# Finalmente, iremos testar nosso modelo com dados específicos de teste e verificaremos os scores de assertividade

previsoes = modelo.predict(X_teste)
accuracy = accuracy_score(y_teste, previsoes)
f1 = f1_score(y_teste, previsoes, average='weighted')
print(f'Acuracia: {accuracy}, F1: {f1}')

Acuracia: 0.8458390177353342, F1: 0.8357400306458583
